
# Update maintainers

Below are some ad-hoc code chunks to assist me in mass-updating the maintainers of the conda-forge R recipes

In [1]:
import os
import subprocess as sp
import sys
import time

In [2]:
# Authenticate with GitHub

import github

gh_token = os.environ['GH_TOKEN']
gh = github.Github(gh_token)
gh_me = gh.get_user()
print("Authenticated as %s"%(gh_me.login))

Authenticated as jdblischak


In [3]:
# Check rate limit
gh.get_rate_limit()

RateLimit(core=Rate(reset=2022-01-03 01:42:52, remaining=4772, limit=5000))

In [4]:
# Search
# https://pygithub.readthedocs.io/en/latest/github.html?highlight=search#github.MainClass.Github.search_code
results = gh.search_code(query="org:conda-forge language:YAML jdblischak conda-forge/r")
results.totalCount

148

In [5]:
# Repositories to keep
skip = ['r-feedstock',
        'r-base-feedstock',
        'r-git2r-feedstock',
        'r-knitr-feedstock',
        'r-rmarkdown-feedstock',
        'r-workflowr-feedstock',
        'r-reshape-feedstock',
        'r-lokern',
        'r-densityclust',
        'r-rtensor']

In [6]:
# Convert search results to dict of repository objects
dict_repo = {}
limit = 10
counter = 0
for r in results:
    repo = r.repository
    name_repo = os.path.basename(repo.full_name)
    if name_repo in skip:
        continue
    if name_repo[:2] != "r-":
        continue
    dict_repo[name_repo] = repo
    counter += 1
    if counter == limit:
        break
    time.sleep(0.1)
print(len(dict_repo))

10


In [7]:
# Fork repositories
for repo in dict_repo.values():
    print(repo.full_name)
    gh_me.create_fork(repo)
    time.sleep(0.5)

conda-forge/r-matlab-feedstock
conda-forge/r-qiimer-feedstock
conda-forge/r-gapminder-feedstock
conda-forge/r-nistunits-feedstock
conda-forge/r-yaml-feedstock
conda-forge/r-drc-feedstock
conda-forge/r-tidyxl-feedstock
conda-forge/r-longitudinal-feedstock
conda-forge/r-hwriter-feedstock
conda-forge/r-ggsci-feedstock


In [8]:
# Clone repositories
for feedstock in dict_repo.keys():
    print(feedstock)
    localdir="/tmp/%s"%(feedstock)
    remote = "git@github.com:jdblischak/%s.git"%(feedstock)
    cmd = ["git", "clone", "--quiet", remote, localdir]
    if not os.path.exists(localdir):
        sp.run(cmd)

r-matlab-feedstock
r-qiimer-feedstock
r-gapminder-feedstock
r-nistunits-feedstock
r-yaml-feedstock
r-drc-feedstock
r-tidyxl-feedstock
r-longitudinal-feedstock
r-hwriter-feedstock
r-ggsci-feedstock


In [9]:
conda_forge_r = {
    "johanneskoester",
    "bgruening",
    "daler",
    "jdblischak",
    "cbrueffer",
    "dbast",
    "dpryan79"
}

In [10]:
# Update meta.yaml
for feedstock in dict_repo.keys():
    print(feedstock)
    file_meta = "/tmp/" + feedstock + "/recipe/meta.yaml"

    # read
    handle_meta = open(file_meta, "r")
    lines_meta = handle_meta.readlines()
    handle_meta.close()

    # write back non-filtered lines
    handle_meta = open(file_meta, "w")
    for line in lines_meta:
        keep = True
        for maintainer in conda_forge_r:
            if "- %s"%(maintainer) in line.lower(): 
                keep = False
        if keep:
            handle_meta.write(line)

    handle_meta.close()
    

r-matlab-feedstock
r-qiimer-feedstock
r-gapminder-feedstock
r-nistunits-feedstock
r-yaml-feedstock
r-drc-feedstock
r-tidyxl-feedstock
r-longitudinal-feedstock
r-hwriter-feedstock
r-ggsci-feedstock


In [11]:
# Update .github/CODEOWNERS
for feedstock in dict_repo.keys():
    print(feedstock)
    file_owners = "/tmp/" + feedstock + "/.github/CODEOWNERS"

    if not os.path.exists(file_owners):
        continue

    # read (it's a single line)
    handle_owners = open(file_owners, "r")
    line_owners = handle_owners.readline()
    handle_owners.close()

    for maintainer in conda_forge_r:
        line_owners = line_owners.replace(" @%s"%(maintainer), "")
        line_owners = line_owners.replace(" @%s"%(maintainer.capitalize()), "")

    # write
    handle_owners = open(file_owners, "w")
    handle_owners.write(line_owners)
    handle_owners.close()

r-matlab-feedstock
r-qiimer-feedstock
r-gapminder-feedstock
r-nistunits-feedstock
r-yaml-feedstock
r-drc-feedstock
r-tidyxl-feedstock
r-longitudinal-feedstock
r-hwriter-feedstock
r-ggsci-feedstock


In [12]:
# Update README.md
for feedstock in dict_repo.keys():
    print(feedstock)
    file_readme = "/tmp/" + feedstock + "/README.md"

    # read
    handle_readme = open(file_readme, "r")
    lines_readme = handle_readme.readlines()
    handle_readme.close()

    # write back non-filtered lines
    handle_readme = open(file_readme, "w")
    for line in lines_readme:
        keep = True
        for maintainer in conda_forge_r:
            if "* [@%s](https://github.com/%s/)"%(maintainer, maintainer) in line.lower(): 
                keep = False
        if keep:
            handle_readme.write(line)

    handle_readme.close()

r-matlab-feedstock
r-qiimer-feedstock
r-gapminder-feedstock
r-nistunits-feedstock
r-yaml-feedstock
r-drc-feedstock
r-tidyxl-feedstock
r-longitudinal-feedstock
r-hwriter-feedstock
r-ggsci-feedstock


In [13]:
%%bash

# Commit updated recipes to branch "update-maintainers"

for repo in /tmp/r-*-feedstock
do
  echo "$repo"
  cd "$repo"
  git checkout -b "update-maintainers"
  git add .github/CODEOWNERS README.md recipe/meta.yaml
  git commit -m "[ci skip] [skip ci] ***NO_CI*** Update maintainers"
done

/tmp/r-drc-feedstock
[update-maintainers 0ebe74b] [ci skip] [skip ci] ***NO_CI*** Update maintainers
 3 files changed, 1 insertion(+), 11 deletions(-)
/tmp/r-gapminder-feedstock
[update-maintainers 0664970] [ci skip] [skip ci] ***NO_CI*** Update maintainers
 3 files changed, 1 insertion(+), 13 deletions(-)
/tmp/r-ggsci-feedstock
[update-maintainers a347822] [ci skip] [skip ci] ***NO_CI*** Update maintainers
 3 files changed, 1 insertion(+), 13 deletions(-)
/tmp/r-hwriter-feedstock
[update-maintainers 8d1e387] [ci skip] [skip ci] ***NO_CI*** Update maintainers
 3 files changed, 1 insertion(+), 13 deletions(-)
/tmp/r-longitudinal-feedstock
[update-maintainers 1130e63] [ci skip] [skip ci] ***NO_CI*** Update maintainers
 3 files changed, 1 insertion(+), 13 deletions(-)
/tmp/r-matlab-feedstock
[update-maintainers 8971d73] [ci skip] [skip ci] ***NO_CI*** Update maintainers
 3 files changed, 1 insertion(+), 13 deletions(-)
/tmp/r-nistunits-feedstock
[update-maintainers 88a9716] [ci skip] [ski

Switched to a new branch 'update-maintainers'
Switched to a new branch 'update-maintainers'
Switched to a new branch 'update-maintainers'
Switched to a new branch 'update-maintainers'
Switched to a new branch 'update-maintainers'
Switched to a new branch 'update-maintainers'
Switched to a new branch 'update-maintainers'
Switched to a new branch 'update-maintainers'
Switched to a new branch 'update-maintainers'
Switched to a new branch 'update-maintainers'


In [14]:
%%bash

# Push to GitHub

for repo in /tmp/r-*-feedstock
do
  echo "$repo"
  cd "$repo"
  git push origin update-maintainers
done

/tmp/r-drc-feedstock
/tmp/r-gapminder-feedstock
/tmp/r-ggsci-feedstock
/tmp/r-hwriter-feedstock
/tmp/r-longitudinal-feedstock
/tmp/r-matlab-feedstock
/tmp/r-nistunits-feedstock
/tmp/r-qiimer-feedstock
/tmp/r-tidyxl-feedstock
/tmp/r-yaml-feedstock


remote: 
remote: Create a pull request for 'update-maintainers' on GitHub by visiting:        
remote:      https://github.com/jdblischak/r-drc-feedstock/pull/new/update-maintainers        
remote: 
To github.com:jdblischak/r-drc-feedstock.git
 * [new branch]      update-maintainers -> update-maintainers
remote: 
remote: Create a pull request for 'update-maintainers' on GitHub by visiting:        
remote:      https://github.com/jdblischak/r-gapminder-feedstock/pull/new/update-maintainers        
remote: 
To github.com:jdblischak/r-gapminder-feedstock.git
 * [new branch]      update-maintainers -> update-maintainers
remote: 
remote: Create a pull request for 'update-maintainers' on GitHub by visiting:        
remote:      https://github.com/jdblischak/r-ggsci-feedstock/pull/new/update-maintainers        
remote: 
To github.com:jdblischak/r-ggsci-feedstock.git
 * [new branch]      update-maintainers -> update-maintainers
remote: 
remote: Create a pull request for 'update-maintainers' on

In [15]:
# Open pull requests
# https://pygithub.readthedocs.io/en/latest/github_objects/Repository.html?highlight=create_pull#github.Repository.Repository.create_pull
# https://docs.github.com/en/rest/reference/pulls#create-a-pull-request
# https://pygithub.readthedocs.io/en/latest/examples/PullRequest.html?highlight=create%20a%20pull%20request#create-a-new-pull-request
for repo in dict_repo.values():
    print(repo.full_name)

    repo.create_pull(
        title="[ci skip] [skip ci] ***NO_CI*** Update maintainers",
        body="Remove individually listed maintainers that are members of the conda-forge/r team",
        base="master",
        head="jdblischak:update-maintainers"
    )

    time.sleep(1)

conda-forge/r-matlab-feedstock
conda-forge/r-qiimer-feedstock
conda-forge/r-gapminder-feedstock
conda-forge/r-nistunits-feedstock
conda-forge/r-yaml-feedstock
conda-forge/r-drc-feedstock
conda-forge/r-tidyxl-feedstock
conda-forge/r-longitudinal-feedstock
conda-forge/r-hwriter-feedstock
conda-forge/r-ggsci-feedstock
